In [ ]:
pip install numpy pandas scikit-learn tensorflow


In [1]:
import json
from zipfile import ZipFile
kaggle_credentails = json.load(open("kaggle.json"))

In [2]:
# setup Kaggle API key as environment variables
import os
os.environ['KAGGLE_USERNAME'] = kaggle_credentails["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentails["key"]

In [3]:
#!/bin/bash
!kaggle datasets download praveengovi/emotions-dataset-for-nlp

Dataset URL: https://www.kaggle.com/datasets/praveengovi/emotions-dataset-for-nlp
License(s): CC-BY-SA-4.0
  0% 0.00/721k [00:00<?, ?B/s]
100% 721k/721k [00:00<00:00, 66.7MB/s]


In [4]:
!ls


emotions-dataset-for-nlp.zip  kaggle.json  sample_data


In [5]:
# Unzip the downloaded dataset
with ZipFile("emotions-dataset-for-nlp.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [6]:
import pandas as pd

# Load dataset
def load_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            text, emotion = line.strip().split(';')
            data.append((text, emotion))
    return pd.DataFrame(data, columns=['Text', 'Emotion'])

# Load train, validation, and test datasets
train_data = load_data('/content/train.txt')
val_data = load_data('/content/val.txt')
test_data = load_data('/content/test.txt')

# Display sample data
print(train_data.head())


                                                Text  Emotion
0                            i didnt feel humiliated  sadness
1  i can go from feeling so hopeless to so damned...  sadness
2   im grabbing a minute to post i feel greedy wrong    anger
3  i am ever feeling nostalgic about the fireplac...     love
4                               i am feeling grouchy    anger


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['Text']).toarray()
X_val = vectorizer.transform(val_data['Text']).toarray()
X_test = vectorizer.transform(test_data['Text']).toarray()

# Encode labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(train_data['Emotion'])
y_val = encoder.transform(val_data['Emotion'])
y_test = encoder.transform(test_data['Emotion'])


In [8]:
# Inspecting the shape of the transformed data
print("Shape of X_train:", X_train.shape)
print("Shape of X_val:", X_val.shape)
print("Shape of X_test:", X_test.shape)

# Convert a sample of the TF-IDF matrix to DataFrame to view features
X_train_df = pd.DataFrame(X_train, columns=vectorizer.get_feature_names_out())
print(X_train_df.head())


Shape of X_train: (16000, 5000)
Shape of X_val: (2000, 5000)
Shape of X_test: (2000, 5000)
   abandon  abandoned  abandonment  abc  abdomen  abide  abilities  ability  \
0      0.0        0.0          0.0  0.0      0.0    0.0        0.0      0.0   
1      0.0        0.0          0.0  0.0      0.0    0.0        0.0      0.0   
2      0.0        0.0          0.0  0.0      0.0    0.0        0.0      0.0   
3      0.0        0.0          0.0  0.0      0.0    0.0        0.0      0.0   
4      0.0        0.0          0.0  0.0      0.0    0.0        0.0      0.0   

   abit  able  ...  yours  yourself  youth  youtube  youve  zealand  zero  \
0   0.0   0.0  ...    0.0       0.0    0.0      0.0    0.0      0.0   0.0   
1   0.0   0.0  ...    0.0       0.0    0.0      0.0    0.0      0.0   0.0   
2   0.0   0.0  ...    0.0       0.0    0.0      0.0    0.0      0.0   0.0   
3   0.0   0.0  ...    0.0       0.0    0.0      0.0    0.0      0.0   0.0   
4   0.0   0.0  ...    0.0       0.0    0.0      0

In [9]:
# Viewing the encoded labels for training data
print("Encoded y_train:", y_train[:10])  # Show the first 10 encoded labels

# Viewing the corresponding emotion labels
print("Original labels for y_train:", encoder.inverse_transform(y_train[:10]))  # Show original emotion labels

# Check label distribution
print("Label distribution in y_train:", pd.Series(y_train).value_counts())


Encoded y_train: [4 4 0 3 0 4 5 1 2 3]
Original labels for y_train: ['sadness' 'sadness' 'anger' 'love' 'anger' 'sadness' 'surprise' 'fear'
 'joy' 'love']
Label distribution in y_train: 2    5362
4    4666
0    2159
1    1937
3    1304
5     572
Name: count, dtype: int64


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Train the model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Validate the model
y_pred_val = model.predict(X_val)
print(classification_report(y_val, y_pred_val, target_names=encoder.classes_))


              precision    recall  f1-score   support

       anger       0.91      0.82      0.87       275
        fear       0.85      0.75      0.80       212
         joy       0.85      0.94      0.89       704
        love       0.89      0.63      0.74       178
     sadness       0.86      0.94      0.90       550
    surprise       0.84      0.58      0.69        81

    accuracy                           0.86      2000
   macro avg       0.87      0.78      0.81      2000
weighted avg       0.86      0.86      0.86      2000



In [12]:


# Expanded emotions and corresponding yoga routines with YouTube links
emotion_to_yoga = {
    "joy": {
        "routine": ["Energizing Yoga: Sun Salutations, Backbends", "Vinyasa Flow"],
        "video_links": [
            "https://youtu.be/1xRX1MuoImw?si=Fx9P_Ad-Yxa0VE2K",  # Sun Salutations Tutorial
            "https://youtu.be/Wix86Uw0cWc?si=leTkwvWDw1ZL3cgs"   # Vinyasa Flow Tutorial
        ]
    },
    "sadness": {
        "routine": ["Calming Yoga: Child's Pose, Forward Folds", "Restorative Yoga: Legs Up the Wall"],
        "video_links": [
            "https://youtu.be/2MJGg-dUKh0?si=hpMb9PxHtG9VBJUA",  # Child's Pose Tutorial
            "https://youtu.be/do_1LisFah0?si=vN19sTl_r_rbtDZh"   # Legs Up the Wall Tutorial
        ]
    },
    "anger": {
        "routine": ["Cooling Yoga: Alternate Nostril Breathing, Seated Twists", "Yin Yoga: Deep Stretching"],
        "video_links": [
            "https://youtu.be/8VwufJrUhic?si=y46S2Y7ZPWBcCSzA",  # Alternate Nostril Breathing
            "https://youtu.be/IYiV4ztPVu0?si=DFVlzNbjvOSyWbmh"   # Yin Yoga Deep Stretching
        ]
    },
    "fear": {
        "routine": ["Grounding Yoga: Tree Pose, Warrior II", "Root Chakra Yoga: Focus on Stability"],
        "video_links": [
            "https://youtu.be/uELr6MPi7pI?si=CjDNg11IdlBSbdX-",  # Tree Pose Tutorial
            "https://youtu.be/0pbuvhbg7po?si=qlfsRle5uHdq9m7h"   # Root Chakra Yoga
        ]
    },
    "love": {
        "routine": ["Heart-Opening Yoga: Cobra, Camel Pose", "Chest and Shoulders Yoga: Poses for Love"],
        "video_links": [
            "https://youtu.be/luTSRGXPEMs?si=UKHa5PhuXD2v2dIP",  # Cobra Pose Tutorial
            "https://youtu.be/CZJVeUD_Ou8?si=pe2Fk0XAMGP4XtA1"   # Chest and Shoulders Yoga Tutorial
        ]
    },
    "stress": {
        "routine": ["Stress Relief Yoga: Seated Forward Bend, Savasana", "Breathing Exercises: Box Breathing"],
        "video_links": [
            "https://www.youtube.com/watch?v=gGV8P4yJhT4",  # Seated Forward Bend Tutorial
            "https://www.youtube.com/watch?v=I02zRE9cV9I"   # Box Breathing Tutorial
        ]
    },
    "anxiety": {
        "routine": ["Gentle Yoga: Cat-Cow Pose, Child's Pose", "Breathing & Relaxation: Pranayama"],
        "video_links": [
            "https://youtube.com/shorts/MeVmhLxrQSk?si=y6Zm4cExmo6NMNsn",  # Cat-Cow Pose Tutorial
            "https://youtu.be/I77hh5I69gA?si=JuimCUuvnTPDm6jr"   # Pranayama Breathing
        ]
    },
    "grief": {
        "routine": ["Healing Yoga: Gentle Spinal Twist, Savasana", "Emotional Release Yoga"],
        "video_links": [
            "https://youtu.be/qEVNj4tcr0Y?si=7QeSLz1y3oxy5pvg",  # Gentle Spinal Twist
            "https://youtu.be/K-0RWe0NdWw?si=eRAGUIHcKcFzDbmd"   # Emotional Release Yoga
        ]
    },
    "excitement": {
        "routine": ["Dynamic Yoga: Chair Pose, Boat Pose", "Flow Yoga: Dynamic Vinyasa"],
        "video_links": [
            "hhttps://youtu.be/qQZOlIHMlmA?si=J3D0fDOEdGLNGTLu",  # Chair Pose Tutorial
            "https://youtu.be/Z6jRKThDCBU?si=8a3Zb6mIfF6xcJNE"   # Vinyasa Flow Tutorial
        ]
    }
}


In [13]:
def recommend_yoga(text):
    text_vector = vectorizer.transform([text]).toarray()
    predicted_emotion = encoder.inverse_transform(model.predict(text_vector))[0]

    # Get the recommended yoga routines and video links
    yoga_info = emotion_to_yoga.get(predicted_emotion, {"routine": ["General Relaxation Yoga"], "video_links": ["https://www.youtube.com"]})

    return yoga_info["routine"], yoga_info["video_links"]


In [14]:

user_input = "I am feeling nervous"
routines, video_links = recommend_yoga(user_input)

print("Recommended Yoga Routines:")
for routine, link in zip(routines, video_links):
    print(f"- {routine}: {link}")



Recommended Yoga Routines:
- Grounding Yoga: Tree Pose, Warrior II: https://youtu.be/uELr6MPi7pI?si=CjDNg11IdlBSbdX-
- Root Chakra Yoga: Focus on Stability: https://youtu.be/0pbuvhbg7po?si=qlfsRle5uHdq9m7h
